<a href="https://colab.research.google.com/github/LordHarsh/Neural_Style_Transfer/blob/main/Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is a trial

In [ ]:
!pip install pytube
!pip install ffmpy

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from pytube import YouTube
import os
import cv2
from PIL import Image
import glob
import ffmpy
from google.colab.patches import cv2_imshow

In [ ]:
!curl https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1200px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg --output style.jpg
!curl https://static01.nyt.com/images/2013/07/06/nyregion/06empire-web/06empire-web-superJumbo.jpg --output content.jpg

In [ ]:
def transfer(content_img_path, style_img_path):
  content_img = plt.imread(content_img_path)
  style_img = plt.imread(style_img_path)

  content_img = content_img.astype(np.float32)[np.newaxis, ...] / 255.0
  style_img = style_img.astype(np.float32)[np.newaxis, ...] / 255.0
  
  style_img =tf.image.resize(style_img, (265, 265))
  hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
  outputs = hub_module(tf.constant(content_img), tf.constant(style_img))
  stylized_image = outputs[0]
  return stylized_image

In [ ]:
def StyleTransferVideo(list_content_img_path, style_img_path):
  content_img_list = [plt.imread(content_img_path) for content_img_path in list_content_img_path]
  style_img = plt.imread(style_img_path)

  content_img_list = [content_img.astype(np.float32)[np.newaxis, ...] / 255.0 for content_img in content_img_list]
  style_img = style_img.astype(np.float32)[np.newaxis, ...] / 255.0
  
  style_img =tf.image.resize(style_img, (265, 265))
  

  hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
  outputs = [hub_module(tf.constant(content_img), tf.constant(style_img)) for content_img in content_img_list]
  print("-----------Style Transfer completed on all images-----------")
  return outputs

In [ ]:
def ExtractFrames(video_path, directory):
  vidcap = cv2.VideoCapture(vidpath)

  success,image = vidcap.read()
  count = 0

  newpath = os.path.join(directory, 'imgfolder')

  if not os.path.exists(newpath):
      os.makedirs(newpath)
  
  while success:
    if (count%30==0):
      cv2.imwrite(os.path.join(newpath, "frame%.6d.jpg" % count), image)     # save frame as JPEG file
    success,image = vidcap.read()
    count += 1 

  frames = [f for f in os.listdir(newpath) if os.path.isfile(os.path.join(newpath, f))]
  frames.sort()
  print("-----------Frame extraction completed-----------")
  return frames, newpath

In [ ]:
def YTDownload(enterlink):
  return YouTube(enterlink).streams.filter(only_video=True, subtype='mp4').order_by('resolution').desc().first().download()

In [ ]:
def ConvertTensorToCV2(generated_tensor):
  # Tensorflow array to PIL image
  generated_pil_img_array = [tf.keras.preprocessing.image.array_to_img(generated_tensor[i][0][0]) for i in range(len(generated_tensor))]

  # PIL image to cv2 image
  generated_cv2_img_array = [np.array(generated_pil_img_array[i], dtype=np.uint8)[:, :, ::-1].copy()  for i in range(len(generated_pil_img_array))]
  print("-----------Image conversion completed-----------")
  return generated_cv2_img_array

In [ ]:
def ConvertImagesToVideo(final_cv2_image_list, length=1):
  height, width, layers = final_cv2_image_list[0].shape
  size = (width,height)

  path = os.path.join(os.getcwd(), 'StyleVideo.avi')
  out = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'DIVX'), length, size)
  
  for i in range(len(final_cv2_image_list)):
    out.write(final_cv2_image_list[i])
  out.release()
  print("-----------Video construction completed-----------")
  return path

In [ ]:
def convert_avi_to_mp4(avi_file_path):
  mp4_path = os.path.join(os.getcwd(), 'StyleVideo.mp4')
  ff = ffmpy.FFmpeg(
      inputs={os.path.basename(avi_file_path): None},
      outputs={os.path.basename(mp4_path): None}
  )
  ff.run()
  print("-----------Video conversion completed-----------")
  return mp4_path

In [ ]:
link=input()
vidpath = YTDownload(link)
current_directory = os.getcwd()
onlyfiles, imgfolderpath = ExtractFrames(vidpath, current_directory)
content_img_path_list = [os.path.join(imgfolderpath, pp) for pp in onlyfiles]
style_image_path = os.path.join(os.getcwd(),'style.jpg')
generated_img_list_output = StyleTransferVideo(content_img_path_list, style_image_path)
final_cv2_image_list = ConvertTensorToCV2(generated_img_list_output)
avi_video_path = ConvertImagesToVideo(final_cv2_image_list)
mp4_video_path = convert_avi_to_mp4(avi_video_path)

In [ ]:
!pip install -U kora
from kora.drive import upload_public
url = upload_public('/content/StyleVideo.mp4')
# then display it
from IPython.display import HTML
HTML(f"""<video src={url} width=500 controls/>""")